In [1]:
%load_ext autotime

In [2]:
import magellan as mg
import pandas as pd

time: 695 µs


In [3]:
mg.init_jvm()

True

time: 121 ms


In [4]:
bwk = mg.read_csv(mg.get_install_path()+'/datasets/books/bowker.csv',
                  dtype={'isbn':pd.np.str},
                  low_memory=False, key='id')

time: 46.2 ms


In [5]:
wal = mg.read_csv(mg.get_install_path()+'/datasets/books/walmart.csv', 
                  dtype={'isbn':pd.np.str},
                  low_memory=False, key='id')

time: 918 ms


In [6]:
# Create Attribute equivalence blocker and block using isbn
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(bwk, wal, 'isbn', 'isbn', ['title', 'author', 'pubYear', 'publisher'], 
                    ['title', 'author', 'pubYear', 'publisher'])

time: 134 ms


In [7]:
# Write candidate set to disk -- end of project stage 2
C.to_csv('books_candset.csv')

True

time: 2.54 ms


In [8]:
# Read candidate set 
C = mg.read_csv('books_candset.csv', ltable=wal, rtable=bwk)

time: 3.27 ms


In [9]:
#plan

# 1. sample candidate set --> S
# 2. label S --> L
# 3. Split L into development I, and evaluation set J
# 4. Select best matcher Y, using I 
# 5. Add triggers to Y ---> Z
# 6. Evaluate Z using J


time: 1.09 ms


In [10]:
# Sample table
S = mg.sample_table(C, 65)

time: 1.91 ms


In [11]:
# label table
L = mg.label_table(S, 'gold')

/scratch/pradap/local/share/anaconda/lib/python2.7/site-packages/Magellan-0.1-py2.7.egg/magellan/gui/mtable_gui.py:29: FutureWarning: iget_value(i, j) is deprecated. Please use .iat[i, j]
  datatable.setItem(i, j, QtGui.QTableWidgetItem(str(tbl.iget_value(i, j))))
/scratch/pradap/local/share/anaconda/lib/python2.7/site-packages/Magellan-0.1-py2.7.egg/magellan/gui/mtable_gui.py:58: FutureWarning: iget_value(i, j) is deprecated. Please use .iat[i, j]
  inp = tbl.iget_value(i, j)


time: 2.56 s


In [12]:
# Load pre-labeled data for demo
L = mg.read_csv('muruga_gold_demo.csv', ltable=bwk, rtable=wal)

time: 3.2 ms


In [13]:
# Split L into development I and J
IJ = mg.train_test_split(L, train_proportion=0.7)
I = IJ['train']
J = IJ['test']

time: 3.36 ms


In [14]:
len(I), len(J)

(45, 20)

time: 1.5 ms


In [15]:
# selecting the best learning matcher
# 1. Create a set of ML-matchers
# 2. Generate features --> feature_table
# 3. Extract features using I and feature_table
# 4. Select best matcher using CV
# 5. Debug the selected matcher (and repeat the above steps)

time: 907 µs


In [16]:
# Create a set of ML-matchers
dt = mg.DTMatcher(name='DecisionTree', random_state=0)
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 2.55 ms


In [17]:
# check name
(dt.name, svm.name)

('DecisionTree', 'SVM_48992931478527666441')

time: 1.56 ms


In [48]:
# Generate features
feature_table = mg.get_features_for_matching(bwk, wal)
# remove id_related features
feature_table = feature_table.ix[3:len(feature_table)]
# restart the numbering
feature_table.reset_index(inplace=True, drop=True)


time: 2.45 s


In [51]:
# select a subset of features
feat_table_subset_1 = feature_table.ix[[0, 3, 4, 6, 23]]

time: 1.45 ms


In [56]:
help(mg.label_table)

Help on function label_table in module magellan.labeler.labeler:

label_table(tbl, col_name, replace=True)
    Label training data
    
    Parameters
    ----------
    tbl : MTable, Table to be labeled
    col_name : String, Name of the label column
    replace : Boolean, specifies whether the column with the given 'col_name' must be overwritten, if it already exists.
    [This option is currently experimental].
    
    Returns
    -------
    result : MTable, Table with labels
    
    Notes
    -----
    The label value is expected to be only 0 or 1.

time: 949 µs


In [54]:
# H = mg.extract_feature_vecs(G, feature_table=feat_table_subset_1, exclude_attrs=[''])

time: 22.3 s
